In [1]:
import json
import numpy as np
import cv2
import os
import shutil
from tqdm.notebook import tqdm
from glob import glob

from writer import COCO_writer
from bbox_utils import (convert_xywh_xyxy, convert_xyxy_xywh)

In [2]:
len(glob('PublicDataset/*.jpg'))

4017

In [3]:
categories = [
    {
        'name': 'person',
        'supercategory': '',
        'id': 0,
    },
    {
        'name': 'vehicle',
        'supercategory': '',
        'id': 1,
    },
    {
        'name': 'fire',
        'supercategory': '',
        'id': 2,
    },
    {
        'name': 'animal',
        'supercategory': '',
        'id': 3,
    },
    {
        'name': 'smoke',
        'supercategory': '',
        'id': 4,
    },
]

In [4]:
id_to_name = {
    0: 'smoke',
    1: 'fire',
}

writer = COCO_writer(categories)


track_id = 0
for image_path in tqdm(glob('PublicDataset/*.jpg')):
    image = cv2.imread(image_path)
    ih, iw = image.shape[:2]
    image_name = os.path.split(image_path)[1]
    txt_path = os.path.splitext(image_path)[0] + '.txt'
    annots = []
    image_id, _ = writer.add_frame(*image.shape[:2], filename=image_name)
    shutil.copyfile(image_path, os.path.join('images', image_name))
    with open(txt_path) as in_file:
        for line in in_file:
            cat, x, y, bw, bh = line.split(' ')
            cat, x, bw, y, bh = int(cat), float(x), float(bw), float(y), float(bh)
            bbox = [
                (x - bw / 2) * iw,
                (y - bh / 2) * ih,
                (x + bw / 2) * iw,
                (y + bh / 2) * ih,
                cat,
            ]
            annots.append(bbox)
    annots = np.array(annots, dtype=np.int32)
#     print(annots)
    for b in annots:
        cv2.rectangle(image, tuple(b[:2]), tuple(b[2:4]), (0, 0, 255), 3)
        cv2.putText(image, id_to_name[b[-1]], tuple(b[:2]),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    1, (255, 255, 255), 4)
        bb = convert_xyxy_xywh(b[:4])
        cat_id = writer.get_cat_id(id_to_name[b[-1]])
        writer.add_annotation(image_id, bb, track_id, cat_id)
        track_id += 1
#     cv2.imshow('image', image)
#     cv2.waitKey(0)
writer.write_result(os.path.join('annotations', 'instances_default.json'))

In [36]:
cv2.destroyAllWindows()